## Why build a belief tree?
- Can use in any context - always-on, online text, etc.
- Makes for far more interesting generative content, closer to how humans would make content.
- Can build stories from the beliefs and changing their state.
- Can establish humour from identifying delta between reality/perceived reality (in beliefs), or conflict between two beliefs.
- Can say strange things that feel right/wrong, built on top of those beliefs.

## Notable findings
- Treating historical account of actual communications between people, then saying an author "wrote it", makes the LLM able to extract coherent beliefs.
- Using actual behaviour as context, instead of derived content gets real beliefs. (what they do and decide instead of what they say they do or prefer). This is why stated beliefs are not as meaningful/useful - 

In [1]:
import os
from openai import OpenAI

PROJECT_ID = "proj_hUizl3mrZGSfmp4C6DI60dJo"
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [40]:
model = 'gpt-4o-mini'
lowlevel_belief_extraction_system_prompt = \
"""
You are an expert in psychology, philosophy, propaganda, and literary analysis. Your task is to extract and classify the beliefs expressed in the following text.

Beliefs can fall into three categories:

True: The belief is assumed or presented as a factual/justified claim.
False: The belief is assumed to be untrue or framed as untrue/mistaken/debunked.
Meaningless: The belief was once seen as true or false, but it has been transcended or rendered irrelevant.
For each belief you identify, provide:

The belief statement: What belief is being expressed?
Category: Is the belief true, false, or meaningless (and why)?
Reasoning: What reasoning (if any) supports or challenges this belief in the text?
Example 1:

Text: It took a long time to go the whole way around the world.

Belief: "The Earth is round."
Category: True
Reasoning: This belief relevant as a shared belief, necessary for the text to have meaning, and is implicit in the text.

Example 2:

Text: We're in a simulation, and since Earth is simply a projection within it, we should be able to travel from one city to another instantly. 

Belief: "The Earth is round."
Category: Meaningless
Reasoning: This belief is commonly held and contentious, and makes the text more profound.

Bad example 1 (don't do this):

Text: The Earth is flat, so you couldn't have gone the whole way around.

Belief: "The Earth is round."
Category: False
Reasoning: This belief explicit, and therefore not what you're looking for.
"""
highlevel_belief_extraction_system_prompt = \
"""
You are an expert in psychology, philosophy, propaganda, and literary analysis. Your task is to extract the beliefs expressed 'between the lines' in the following text.

The belief statement: What belief is being expressed 'between the lines'?
Reasoning: Why is this belief important to the author of the text?
"""
first_layer_extraction_prompt = "Extract the implicit beliefs in the following text and number them:"
next_layer_extraction_prompt = "The following text is a list of beliefs. Extract the beliefs that these beliefs depend upon and for each extracted belief, list the id numbers of the beliefs that depend upon it."


In [44]:
def extract_beliefs_request(text: str, text_context: str = None, layer:int=1, level: str = 'lowlevel'):
    messages = []
    if level == 'lowlevel':
        messages.append({
            "role": "system",
            "content": lowlevel_belief_extraction_system_prompt
        })
    else:
        messages.append({
            "role": "system",
            "content": highlevel_belief_extraction_system_prompt
        })
    if text_context:
        messages.append({
            "role": "user",
            "content": "Here is the context of the text: " + text_context
        })
    messages += [
        {
            "role": "user",
            "content": first_layer_extraction_prompt if layer==1 else next_layer_extraction_prompt
        },
        {
            "role": "user",
            "content": text
    }]
    response = client.chat.completions.create(model=model, messages=messages)
    return response.to_dict()['choices'][0]['message']['content']

In [48]:
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader('/Users/douglashindson/Documents/James White.pdf')

# printing number of pages in pdf file
print(len(reader.pages))

# creating a page object
page = reader.pages[0]

# extracting text from page
text = page.extract_text()
print(len(text))
print(text[:3000])
print(text[-100:])

Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)


9
8029
iMessageWed, Oct 2 at 08:29Hey, this is Douglas Hindson. I’m going to be in San Francisco for 2wks from the 13th Oct. I’d enjoy hanging out while I’m there. I’m staying with one of my university friends in the city.Wed, Oct 2 at 11:02Hello Douglas Hindson. Are you going to be in South Bay at all? If not, I can come up to SF whenever since it’s not too far.Ha, yes, excessively formal. I’ve never seen my full name in casual conversation before, feels bot-like. I don’t know the lay of the land, is that a region in LA or a region of san fran?Wed, Oct 2 at 17:34Haha yeah I don’t think I’ve ever heard someone call you Douglas. South Bay is closer to San Jose area. I live/work in Mountain View, but go up to SF fairly regularly since its pretty close and that’s usually where events are Wed, Oct 2 at 19:40I changed to Douglas when I left Canada, only Canadian people call me Doug, and you’re now nominally American.Ok cool, I have some plans, but lots of flexibility, so if there’s cool thi

In [32]:
result = extract_beliefs_request(text, text_context = "This is a conversation between two friends.")
result

In [45]:
result = extract_beliefs_request(text, text_context = "This is a conversation between two friends.", level='highlevel')
from pprint import pprint
pprint(result)

('1. **Implicit Belief**: The author values flexibility and spontaneity in '
 'social planning.\n'
 '   - **Reasoning**: This belief is reflected in the back-and-forth '
 'discussions about available days and activities, emphasizing a willingness '
 'to adjust plans based on mutual interests and opportunities. It highlights '
 'the importance of adaptability in maintaining friendships and enjoying '
 'experiences together.\n'
 '\n'
 '2. **Implicit Belief**: The author appreciates casual, informal '
 'communication over excessive formality.\n'
 '   - **Reasoning**: The initial reaction to the use of the full name '
 '"Douglas" underscores a preference for a relaxed and friendly tone in '
 'conversation. This suggests that they find value in authenticity and a sense '
 'of ease in interactions, which can strengthen personal connections.\n'
 '\n'
 '3. **Implicit Belief**: The author believes in the importance of shared '
 'experiences for building camaraderie.\n'
 '   - **Reasoning**: The